In [16]:
import redis
import json
import time

# Connect to Redis
r = redis.Redis(
    host='redis',  # or your Redis server's hostname
    port=6379,         # default Redis port
)

In [9]:
r.keys('*')

[b'saq:default:schedule',
 b'saq:default:sweep',
 b'saq:default:stats:10a92a7c-cb48-11ee-9d2c-0242ac1d0004',
 b'saq:default:stats:10ac59ae-cb48-11ee-b9cf-0242ac1d0004',
 b'saq:default:stats:10a91b86-cb48-11ee-ada5-0242ac1d0004',
 b'saq:default:stats',
 b'saq:default:incomplete',
 b'saq:job:default:cron:handle_finished_calcs']

In [3]:
for key in r.scan_iter("saq:job:default:*"):
    print(key)

b'saq:job:default:cron:handle_finished_calcs'


In [14]:
stats_data = {}

for key in r.scan_iter("saq:job:default:*"):
    # Determine the type of data stored at the key
    key_type = r.type(key).decode('utf-8')
    value = r.get(key)
    
    # Decode the key (if necessary) and store the retrieved data
    decoded_key = key.decode('utf-8')
    stats_data[decoded_key] = value

    for key, value in stats_data.items():
        data_str = value.decode('utf-8')
        data_dict = json.loads(data_str)

        if (data_dict['status'] == 'complete'):
            print(f"Job {key} completed")
        elif (data_dict['status'] == 'failed'):
            print(f"Job {key} failed")
        elif (data_dict['status'] == 'active'):
            print(f"Job {key} active")

Job saq:job:default:8fe3167c-cb48-11ee-ae11-0242ac1d0003 active
Job saq:job:default:8fe3167c-cb48-11ee-ae11-0242ac1d0003 active


In [22]:
time.time() * 1000

1707922469183.0898

In [1]:
24 * 60 * 60

86400

In [25]:
161182 / 1000

161.182

In [21]:
time.time() * 1000 - 1707922300719

161182.79638671875

In [15]:
stats_data

{'saq:job:default:8fe3167c-cb48-11ee-ae11-0242ac1d0003': b'{"function": "calculate_piece", "kwargs": {"ui_id": "d4ed5b10-2f1a-4c2a-a4e8-9a2d27a344af"}, "queue": "default", "key": "8fe3167c-cb48-11ee-ae11-0242ac1d0003", "timeout": 172800, "retries": 3, "attempts": 1, "queued": 1707922300717, "started": 1707922300719, "touched": 1707922300719, "status": "active"}',
 'saq:job:default:cron:handle_finished_calcs': b'{"function": "handle_finished_calcs", "queue": "default", "key": "cron:handle_finished_calcs", "scheduled": 1707922320, "queued": 1707922260753, "status": "queued"}'}

In [12]:
data_bytes = stats_data['saq:default:stats:10ac59ae-cb48-11ee-b9cf-0242ac1d0004']
data_str = data_bytes.decode('utf-8')
data_dict = json.loads(data_str)

KeyError: 'saq:default:stats:10ac59ae-cb48-11ee-b9cf-0242ac1d0004'

In [7]:
data_dict

{'function': 'handle_finished_calcs',
 'queue': 'default',
 'key': 'cron:handle_finished_calcs',
 'scheduled': 1707922140,
 'queued': 1707922080418,
 'status': 'queued'}

In [5]:
next_task = r.lindex('my_queue', 0)

In [6]:
next_task